In [1]:
from datasets import load_dataset

/home/pulsar/miniconda3/envs/search-engine/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
from datasets import load_dataset
import pandas as pd
dataset = load_dataset("ms_marco", "v1.1")
    

In [43]:
def df_to_list(df):
    queries = df['query'].tolist()
    passage_container = df['passages'].tolist()
    passage_texts = []
    for item in passage_container:
        for passage_text in item["passage_text"]:
            passage_texts.append(passage_text)

    unique_passage_list = list(set(passage_texts))
    return queries, unique_passage_list
    
def count_unique_words(file_path):
    with open(file_path, 'r') as file:
        content = file.read()
        words = content.split()
        unique_words = set(words)
        return len(unique_words)


In [44]:
val_data = dataset["validation"]
val_df = pd.DataFrame(val_data)

In [45]:
train_data = dataset["train"]
train_df = pd.DataFrame(train_data)

In [46]:
test_data = dataset["test"]
test_df = pd.DataFrame(test_data)



In [7]:
queries_train, passagges_train = df_to_list(train_df)
queries_val, passagges_val = df_to_list(val_df)
queries_test, passagges_test = df_to_list(test_df)

with open('corpus.txt', 'w') as file:
    for query in queries_train:
        file.write(f'{query}\n')
    for passage in passagges_train:
        file.write(f'{passage}\n')
        
    for query in queries_val:
        file.write(f'{query}\n')
    for passage in passagges_val:
        file.write(f'{passage}\n')
        
    for query in queries_test:
        file.write(f'{query}\n')
    for passage in passagges_test:
        file.write(f'{passage}\n')

In [8]:
unique_word_count = count_unique_words('corpus.txt')

In [ ]:
import sentencepiece as spm

model_prefix = 'tokenizer'  # Prefix for the model files
vocab_size = 16000  # Number of tokens in the vocabulary
model_type = 'bpe'  # You can choose 'unigram' or 'bpe'

# Train the SentencePiece model
spm.SentencePieceTrainer.train(
    input='corpus.txt',
    model_prefix=model_prefix,
    vocab_size=vocab_size,
    model_type=model_type
)


In [9]:
print(unique_word_count)

1048501


In [11]:
# Load the trained SentencePiece model
sp = spm.SentencePieceProcessor()
sp.load(f'{model_prefix}.model')

with open('corpus.txt', 'r') as file:
    # Read the entire contents of the file into a string
    file_contents = file.read()

# Now, file_contents contains the text from the file as a string


tokens = sp.encode_as_pieces(file_contents)

In [14]:
print(len(tokens))
token_id = 1
token_dictionary_valid = dict()
token_dict = {}

for token in tokens:
    #token_dictionary_valid[str(token_id)] = token
    token_dict[token] = token_id
    token_id +=1
print(len(token_dict))

# Tokenize a sentence
""" query_tokens = dict()
passage_tokens = dict()
token_id = 0
for query in queries:
    one_query_tokens = sp.encode_as_pieces(query)
    for token in one_query_tokens:
        query_tokens[token] = token_id
        token_id +=1
    
token_id = 0
for unique_text in unique_passage_list:
    one_passage_tokens = sp.encode_as_pieces(unique_text)
    for token in one_passage_tokens:
        passage_tokens[token] = token_id
        token_id +=1
"""



197729583
15705


' query_tokens = dict()\npassage_tokens = dict()\ntoken_id = 0\nfor query in queries:\n    one_query_tokens = sp.encode_as_pieces(query)\n    for token in one_query_tokens:\n        query_tokens[token] = token_id\n        token_id +=1\n    \ntoken_id = 0\nfor unique_text in unique_passage_list:\n    one_passage_tokens = sp.encode_as_pieces(unique_text)\n    for token in one_passage_tokens:\n        passage_tokens[token] = token_id\n        token_id +=1\n'

In [47]:
def df_to_list(df):
    queries = df['query'].tolist()
    passage_container = df['passages'].tolist()
    passage_texts = []
    for item in passage_container:
        for passage_text in item["passage_text"]:
            passage_texts.append(passage_text)

    unique_passage_list = list(set(passage_texts))
    return queries, unique_passage_list

In [48]:
    queries = df['query'].tolist()

    passage_container = df['passages'].tolist()
    
    print(len(queries))
    print(len(passage_container))

NameError: name 'df' is not defined

In [56]:
import random

def construct_unique_passage_list(df):
    passage_texts = []
    for item in passage_container:
        for passage_text in item["passage_text"]:
            passage_texts.append(passage_text)

    return list(set(passage_texts))

def text_to_tokenid(text):
    token_list = sp.encode_as_pieces(text)
    tokenid_list = []
    for token in token_list:
        if token in token_dict:
            tokenid_list.append(token_dict[token])
    return tokenid_list

def construct_tokenid_lists(text_list):
    tokenids_list = []
    for text in text_list:
        tokenids_list.append(text_to_tokenid(text))
    return tokenids_list

def list_to_labelled_dataset(df):
    queries = df['query'].tolist()

    passage_container = df['passages'].tolist()
    
    passage_texts = []
    for item in passage_container:
        for passage_text in item["passage_text"]:
            passage_texts.append(passage_text)
    unique_passage_list = list(set(passage_texts))
  #  print(len(unique_passage_list))
    
    query_tokens = []
    passage_tokens = []
    labels = []
    
    counter = 0
    for item in passage_container:
        for passage_text in item["passage_text"]:
            
            query_tokens.append(queries[counter])
            
            passage_tokens.append(passage_text)
            
            labels.append(1)
        
        for _ in item["passage_text"]:
            
            query_tokens.append(queries[counter])
            
            random_integer = random.randint(0, len(unique_passage_list)-1)
            #print(random_integer)
            passage_tokens.append(unique_passage_list[random_integer])
            
            labels.append(-1)
        
        counter+= 1
        
    query_tokenid_list = construct_tokenid_lists(query_tokens)
    passage_tokenid_list = construct_tokenid_lists(passage_tokens)    
       
    print(len(passage_tokens))
    print(passage_tokens[0])   
    print(len(query_tokens))   
    print(query_tokens[0]) 
    print(len(labels))   
    print(labels[0])  
        
    return unique_passage_list, query_tokenid_list, passage_tokenid_list, labels

In [57]:
unique_passage_list_train, query_tokenid_list_train, passage_tokenid_list_train, train_labels = list_to_labelled_dataset(train_df)


1352386
Since 2007, the RBA's outstanding reputation has been affected by the 'Securency' or NPA scandal. These RBA subsidiaries were involved in bribing overseas officials so that Australia might win lucrative note-printing contracts. The assets of the bank include the gold and foreign exchange reserves of Australia, which is estimated to have a net worth of A$101 billion. Nearly 94% of the RBA's employees work at its headquarters in Sydney, New South Wales and at the Business Resumption Site.
1352386
what is rba
1352386
1


In [58]:
unique_passage_list_val, query_tokenid_list_val, passage_tokenid_list_val, val_labels  = list_to_labelled_dataset(val_df)


164720
The average Walgreens salary ranges from approximately $15,000 per year for Customer Service Associate / Cashier to $179,900 per year for District Manager. Average Walgreens hourly pay ranges from approximately $7.35 per hour for Laboratory Technician to $68.90 per hour for Pharmacy Manager. Salary information comes from 7,810 data points collected directly from employees, users, and jobs on Indeed.
164720
walgreens store sales average
164720
1


In [59]:
unique_passage_list_test, query_tokenid_list_test, passage_tokenid_list_test, test_labels  = list_to_labelled_dataset(test_df)



158352
We have been feeding our back yard squirrels for the fall and winter and we noticed that a few of them have missing fur. One has a patch missing down his back and under both arms. Also another has some missing on his whole chest. They are all eating and seem to have a good appetite.
158352
does human hair stop squirrels
158352
1


In [60]:
unique_passages_tokenids_list = construct_tokenid_lists(unique_passage_list_train)

In [61]:
print(len(query_tokenid_list_train))
print(len(passage_tokenid_list_train))
print(len(train_labels))

1352386
1352386
1352386


In [65]:
import csv 

def export_list_of_lists(filename, data):
    with open(filename, "w", newline="") as file:
        writer = csv.writer(file)
        writer.writerows(data)
        
def export_to_csv(filename, data):
    with open(filename, "w", newline="") as file:
        writer = csv.writer(file)
        writer.writerow(data)

# Example usage:





In [66]:
export_to_csv("train_labels.csv", train_labels)        
export_list_of_lists("train_query_token_ids.csv", query_tokenid_list_train)
export_list_of_lists("train_document_token_ids.csv", passage_tokenid_list_train)

In [67]:

export_list_of_lists("val_query_token_ids.csv", query_tokenid_list_val)
export_list_of_lists("val_document_token_ids.csv", passage_tokenid_list_val)
export_to_csv("val_labels.csv",  val_labels)
export_list_of_lists("test_query_token_ids.csv", query_tokenid_list_test)
export_list_of_lists("test_document_token_ids.csv" , passage_tokenid_list_test)
export_to_csv("test_labels.csv" ,  test_labels )


In [68]:
export_list_of_lists("cached_document_token_ids.csv", unique_passages_tokenids_list)